In [2]:
import json
import os
import sys
sys.path.append('../..')
os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'
import logging
from typing import List, Union, Tuple
from grammar.llm import AnyOpenAILLM
from rag_systems.retrieval.retrieval import Qdrant, KeywordMatching
from grammar.llm import gen_model

logging.basicConfig(filename='log.txt', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# read documents from benchmarks/aurp/aurp-database/documents.json
documents = json.load(open('spider-database/company_documents.json'))

In [49]:
retrieval = KeywordMatching(documents)
def retrieve(query: str, answer:  Union[List, Tuple] = None, true_documents_id:int=None, retrieved_documents=None):
    result = {
        "query": query, 
        "answer": answer, 
        "true_documents_id": true_documents_id,
        "retrieved_documents": [], 
        "gpt_response": None, 
        "retrieval_failed": 0
    }
    # 1 - Retrieval
    if not retrieved_documents:
        items = retrieval.search(query)
        print(len(items))
        result['retrieved_documents_id'] = retrieval.rank(items)[0].id
        retrieved_documents = """### Sources:\n"""+retrieval.format_sources(items)
    result["retrieved_documents"] = retrieved_documents + "\n\n### Question:\n" + query

    # 2 - Check if the answer exists in the retrieved documents
    if int(result['retrieved_documents_id'])!=int(result['true_documents_id']):
        print("Retrieval failed")
        result['retrieval_failed'] += 1
    return result

gen_model.temperature = 0
def retrieval_augmented_response(retrieval_result):
    # 3 - Call OpenAI Chat API to generate a response
    retrieval_result["gpt_response"] = gen_model(retrieval_result["retrieved_documents"], temperature=0)

    return retrieval_result

def check_answer(query, true_answer:  Union[List, Tuple], gpt_response):
    prompt = """Evaluate the accuracy of the given response in relation to the true answer for the specified query. After evaluating, provide a judgement as either "Correct" or "Incorrect" based on whether the ##Given Response## accurately matches the ##True Answer##.

    ##Query##: {}
    ##True Answer##: {}
    ##Given Response##: {}
    ##Judgement##:
    """.format(query, json.dumps(true_answer), gpt_response)
    gpt4_llm = AnyOpenAILLM(model_name = "gpt4-short")
    return gpt4_llm(prompt)


In [50]:
# parameters
linguistic_control = "long"
file_name = f'results_{linguistic_control}'

# load evaluation data
with open(f'QADataGenerator/{linguistic_control}_with_ids.json') as f:
    answers_to_text_queries = json.load(f)

# reduce the dimension over SQL templates 
semantics_groups = {answer: query_list for i in range(len(answers_to_text_queries)) for answer, query_list in answers_to_text_queries[i]}


In [51]:
# evaluate
results = [] 
group_tag = -1

for answer, query_list in semantics_groups.items():
    group_tag += 1
    answer = eval(answer)[0]  # only 1 answer; no query multiplicity
    answer_txt = answer[0] 
    if len(answer) == 2:
        true_documents_id = int(answer[1])
    elif len(answer) == 3:
        true_documents_id = ''.join([str(i) for i in answer[1:]])
        true_documents_id = int(true_documents_id)
        print("=======================")
        print(true_documents_id)
    else:
        print(answer)
        raise ValueError("Invalid answer_from format")
    for query in query_list:
        result = retrieve(query, answer_txt, true_documents_id=true_documents_id)
        if result['retrieval_failed']:
            result["judgement"] = "Incorrect"
        else:
            pass
            # result = retrieval_augmented_response(result)
            # result["judgement"] = check_answer(result["query"], result["answer"], result["gpt_response"])
        result['query_tag'] = group_tag
        results.append(result)    

19
Retrieval failed
19
Retrieval failed
19
Retrieval failed
19
19
19
19
19
Retrieval failed
19
19
Retrieval failed
19
19
19
19
Retrieval failed
19
19
Retrieval failed
19
19
19
Retrieval failed
19
19
19
Retrieval failed
19
Retrieval failed
19
19
Retrieval failed
19
19
19
Retrieval failed
19
19
19
Retrieval failed
19
Retrieval failed
19
Retrieval failed
19
Retrieval failed
19
19
19
19
Retrieval failed
19
Retrieval failed
19
Retrieval failed
19
19
19
Retrieval failed
19
Retrieval failed
19
19
Retrieval failed
19
19
Retrieval failed
19
Retrieval failed
19
19
19
Retrieval failed
19
Retrieval failed
19
Retrieval failed
19
Retrieval failed
19
Retrieval failed
19
19
Retrieval failed
19
Retrieval failed
19
Retrieval failed
19
19
Retrieval failed
19
Retrieval failed
19
19
19
Retrieval failed
19
19
19
Retrieval failed
19
19
19
19
Retrieval failed
19
Retrieval failed
19
Retrieval failed
19
19
Retrieval failed
19
Retrieval failed
19
19
Retrieval failed
19
Retrieval failed
19
19
Retrieval failed
19


In [52]:
# save results
with open(f'eval_results/{file_name}.json', 'w') as f:
    json.dump(results, f, indent=4)


Retrieval failed in 120 out of 171 examples


Retrieval failed in 111 out of 171 examples
